<a href="https://colab.research.google.com/github/suryansh29/DL-project/blob/main/notebook_aditya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install transformers
!pip install datasets
!pip install trl
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 17.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 11.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-ma

In [3]:
from datasets import load_dataset, ClassLabel

mnli = load_dataset("nyu-mll/multi_nli")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
num_labels = 2
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")
model = AutoModelForSequenceClassification.from_pretrained("facebook/opt-350m", num_labels=num_labels)

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

Some weights of OPTForSequenceClassification were not initialized from the model checkpoint at facebook/opt-350m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from datasets import ClassLabel

mnli = mnli.filter(lambda example: example["label"] != 1 and example["label"] != -1)
def change_label(example):
    if example["label"] == 2:
        example["label"] = 1
    return example

mnli = mnli.map(change_label)

Filter:   0%|          | 0/392702 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9815 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9832 [00:00<?, ? examples/s]

Map:   0%|          | 0/261802 [00:00<?, ? examples/s]

Map:   0%|          | 0/6692 [00:00<?, ? examples/s]

Map:   0%|          | 0/6703 [00:00<?, ? examples/s]

In [6]:
from transformers import DataCollatorWithPadding

def tokenize_function(example):
    return tokenizer(example["premise"], example["hypothesis"], truncation=True)

tokenized_datasets = mnli.map(tokenize_function, batched=True).shuffle(seed=42)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/261802 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/6692 [00:00<?, ? examples/s]

Map:   0%|          | 0/6703 [00:00<?, ? examples/s]

In [7]:
from datasets import load_metric
import numpy as np

def compute_metrics(eval_preds):
  metric = load_metric('accuracy')
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

#Evaluating zero shot learing

In [24]:
model_zero_shot = AutoModelForSequenceClassification.from_pretrained("facebook/opt-350m", num_labels=num_labels)

trainer_zero_shot = Trainer(
    model=model_zero_shot,
    eval_dataset=tokenized_datasets["validation_matched"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


trainer_zero_shot.evaluate()

Some weights of OPTForSequenceClassification were not initialized from the model checkpoint at facebook/opt-350m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


{'eval_loss': 1.3192510604858398,
 'eval_accuracy': 0.49506873879258817,
 'eval_runtime': 25.2821,
 'eval_samples_per_second': 264.693,
 'eval_steps_per_second': 33.106}

# Without LORA

In [17]:
model_unfreze_last_layer = AutoModelForSequenceClassification.from_pretrained("facebook/opt-350m", num_labels=num_labels)

for name, param in model_unfreze_last_layer.named_parameters():
  if not name.startswith("model.decoder.layers.23"):
    param.requires_grad = False


training_args_unfreze_last_layer = TrainingArguments("mnli-trainer",
                                  evaluation_strategy="steps",
                                  per_device_eval_batch_size=16,
                                  per_device_train_batch_size=16,
                                  eval_steps=300,
                                  save_steps=7915,
                                  learning_rate=5e-5,
                                  label_smoothing_factor=0.1,
                                  fp16=True,
                                  num_train_epochs=1)

trainer_unfreze_last_layer = Trainer(
    model=model_unfreze_last_layer,
    args=training_args_unfreze_last_layer,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation_matched"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


trainer_unfreze_last_layer.train()

Some weights of OPTForSequenceClassification were not initialized from the model checkpoint at facebook/opt-350m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Accuracy
300,No log,0.616034,0.695457
600,0.658400,0.550200,0.750149
900,0.658400,0.530041,0.767932
1200,0.548500,0.532411,0.770472
1500,0.519800,0.515052,0.789749
1800,0.519800,0.490969,0.800956
2100,0.511600,0.479952,0.814405
2400,0.511600,0.475062,0.817244
2700,0.477700,0.472598,0.816946
3000,0.480700,0.473806,0.824716


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric 

TrainOutput(global_step=16363, training_loss=0.4619123252193043, metrics={'train_runtime': 1681.8355, 'train_samples_per_second': 155.664, 'train_steps_per_second': 9.729, 'total_flos': 3.703124861878272e+16, 'train_loss': 0.4619123252193043, 'epoch': 1.0})

In [22]:
trainer_unfreze_last_layer.save_model('models/unfreeze_last_layer')

from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')




model_dir = '/content/drive/My Drive/Colab Notebooks/models/'
trainer_unfreze_last_layer.save_model(model_dir + 'unfreze_last_layer/model')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 5.4 MB/s eta 0:00:00


In [11]:
from peft import LoraConfig, TaskType, get_peft_model

peft_config = LoraConfig(task_type= TaskType.SEQ_CLS, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)


peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()
"output: trainable params: 2359296 || all params: 1231940608 || trainable%: 0.19151053100118282"

training_args = TrainingArguments(
    output_dir="suryansh/dl-project/mt0-large-lora",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    eval_steps=300,
    save_steps=7915,
    learning_rate=5e-5,
    fp16=True,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainable params: 787,456 || all params: 331,984,896 || trainable%: 0.2371963331729405


In [12]:
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation_matched"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.231400,0.214178,0.917065


<ipython-input-7-07bb5a71622b>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('accuracy')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


TrainOutput(global_step=16363, training_loss=0.2778023487415814, metrics={'train_runtime': 1811.7256, 'train_samples_per_second': 144.504, 'train_steps_per_second': 9.032, 'total_flos': 3.712737393967104e+16, 'train_loss': 0.2778023487415814, 'epoch': 1.0})

In [18]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')




model_dir = '/content/drive/My Drive/Colab Notebooks/models/'
trainer.save_model(model_dir + 'lora_opt/model')

In [20]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


{'eval_loss': 0.2141784131526947,
 'eval_accuracy': 0.917065152420801,
 'eval_runtime': 18.3865,
 'eval_samples_per_second': 363.962,
 'eval_steps_per_second': 22.788,
 'epoch': 1.0}